In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [2]:
#Loading the dataset
email_data=pd.read_csv("sms_raw_NB.csv",encoding="ISO-8859-1")
#These are in test form,open the data frame there are ham or spam

In [3]:
#Cleaning the data
#The function tokenize the text and removes words with fewer than 4 characters
import re
def cleaning_text(i):
    i=re.sub("[^A-Z a-z""]+","",i).lower()
    w=[]
    #every thing else A to Z and a to z is going to space andabs
    #we will take each row and tokenize
    for word in i.split(""):
        if len(word)>3:
            w.append(word)
    return("",join(w))
    #Testing above function with sample text
    cleaning_text("Hope you are having a good week. Just checking in")
    cleaning_text("hope i can understand your felling12321.123.hi how are you")
    cleaning_text("Hi how are you")
#Note the ddataframe size is 5559,2 now removing spaces

In [4]:
#Removing empty rows
email_data=email_data.loc[email_data.text!="",:]
email_data.shape

(5559, 2)

In [5]:
#You can use count count vectorizer which directly converts a colection of document
#First we will split the data
from sklearn.model_selection import train_test_split
email_train,email_test=train_test_split(email_data,test_size=0.2)

In [6]:
#Splits each email into a list of words.
#Creating matrix of token count for entire dataframe
def split_into_words(i):
    return[word for word in i.split(" ")]

In [7]:
#defining the preparation of email text into word count matrix  format
#CountVectorizer: converts the emails into a matrix of token counts
#.fit():learns the vocabulary from the text ddata
#.transform():converts the text data to token count matrix
email_bow = CountVectorizer(analyzer=split_into_words).fit(email_data.text)

In [12]:
#Defining bow 
all_emails_matrix = email_bow.transform(email_data.text)
train_email_matrix = email_bow.transform(email_train.text)

#For testing messages
test_email_matrix = email_bow.transform(email_test.text)

#learning term weighting and normalizing entire emails
tfidf_transformer = TfidfTransformer().fit(all_emails_matrix)

#Preparing TFIDF for train mails
train_tfidf = tfidf_transformer.transform(train_email_matrix)
train_tfidf.shape

test_tfidf = tfidf_transformer.transform(test_email_matrix)
test_tfidf.shape


(1112, 15522)

In [17]:
###Now apply to Naive Bayes 
from sklearn.naive_bayes import  MultinomialNB as MB
classifier_mb=MB()
classifier_mb.fit(train_tfidf,email_train.type)
# email_train.type : this is the column in training dataset
# (email_train ) that contains the target labels
# which specify wheather each message is spam or ham (non - spam).
#The .type attibute refers to that specific column in the email_train datframe
# training data prepared in term of tfidf and labels of corresponding training data

MultinomialNB()

In [19]:
#Evaluation on test data
test_pred_m=classifier_mb.predict(test_tfidf)

In [21]:
#Calculate accuracy
accuracy_test_m=np.mean(test_pred_m==email_test.type)
accuracy_test_m

0.954136690647482

In [23]:
#Evaluation on test data accuracy matrix
from sklearn.metrics import accuracy_score
accuracy_score(test_pred_m,email_test.type)
pd.crosstab(test_pred_m,email_test.type)

type,ham,spam
row_0,,
ham,978,51
spam,0,83


In [25]:
#Training data accuracy
train_test_m=classifier_mb.predict(train_tfidf)

# calculate accuracy 
accuracy_train_m = np.mean(test_pred_m == email_test.type)
accuracy_train_m

0.954136690647482

In [27]:
#Accuracy after tuning
#from sklearn.metrics import accuracy_score
#test_pred_lap = classifier_lap.predict(test_tfidf)
#accuracy_score(test_pred_lap,email_test.type)
#pd.crosstab(test_pred_lap,email_test.type)

#accuracy_score


In [29]:
# test data( with Laplace Smoothing ): This accuracy is
# computed after applying Laplace smoothing (with alph = 3)
# to the Naive Bayes model.

# Interpretation : smoothing helps avoid issues when encountering 
# words in the text data that were not seen in the training data 
# (zero -frequncy problem)

classifier_mb_lab = MB(alpha= 3)
classifier_mb_lab.fit(train_tfidf, email_train.type)


MultinomialNB(alpha=3)

In [31]:
# accuracy after tunning 

test_pred_lab = classifier_mb_lab.predict(test_tfidf)
accuracy_test_lab = np.mean(test_pred_lab == email_test.type)
accuracy_test_lab
accuracy_score(test_pred_lab, email_test.type)

0.8911870503597122

In [33]:
from sklearn.metrics import accuracy_score
accuracy_score(test_pred_lab, email_test.type)
pd.crosstab(test_pred_lab, email_test.type)

type,ham,spam
row_0,,
ham,978,121
spam,0,13


In [35]:
#Training data accuracy
train_pred_lap=classifier_mb_lab.predict(train_tfidf)
accuracy_train_lap=np.mean(train_pred_lap==email_train.type)
accuracy_train_lap

0.8855408140319316